In [38]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
from uuid import uuid4

In [40]:
# schemas
nodes_schema = """
CREATE TABLE IF NOT EXISTS nodes (
    id INT PRIMARY KEY
)
"""
names_schema = """
CREATE TABLE IF NOT EXISTS names (
    id TEXT PRIMARY KEY,
    tax_id INT,
    name TEXT,
    FOREIGN KEY (tax_id) REFERENCES nodes(id)
)
"""
# we want a stable mapping from tax_id to name. The other one prioritizes completeness
# for translating name to tax_id
names_unique_schema = """
CREATE TABLE IF NOT EXISTS names_unique (
    id TEXT PRIMARY KEY,
    tax_id INT UNIQUE,
    name TEXT,
    FOREIGN KEY (tax_id) REFERENCES nodes(id)
)
"""
parents_schema = """
CREATE TABLE IF NOT EXISTS parents (
    id TEXT PRIMARY KEY,
    tax_id INT UNIQUE,
    t_kingdom INT NULL,
    t_phylum INT NULL,
    t_class INT NULL,
    t_order INT NULL,
    t_family INT NULL,
    t_genus INT NULL,
    FOREIGN KEY (t_kingdom) REFERENCES nodes(id),
    FOREIGN KEY (t_phylum) REFERENCES nodes(id),
    FOREIGN KEY (t_class) REFERENCES nodes(id),
    FOREIGN KEY (t_order) REFERENCES nodes(id),
    FOREIGN KEY (t_family) REFERENCES nodes(id),
    FOREIGN KEY (t_genus) REFERENCES nodes(id)
)
"""

In [7]:
def create_table(name, schema, cursor, connection):
    cursor.execute(f"DROP TABLE IF EXISTS {name};")
    connection.commit()
    cursor.execute(schema)
    connection.commit()

db_path = Path('../db/taxonomy.db')
dmp_base_path = Path('../taxonomy')

In [8]:
nodes_df = pd.read_csv(dmp_base_path / 'nodes.dmp', header=None, sep="|", usecols=[0, 1, 2], names = [
    'tax_id', 'parent_tax_id', 'rank'
], skipinitialspace=True, converters={'rank': lambda x: x.strip()})

names_df = pd.read_csv(dmp_base_path / 'names.dmp', header=None, sep="|", usecols=[0, 1], names = [
    'tax_id', 'name'
], skipinitialspace=True, converters={'name': lambda x: x.strip()})



In [9]:
# make nodes table
with sqlite3.connect(db_path, autocommit=False) as connection:
    cursor = connection.cursor()
    create_table('nodes', nodes_schema, cursor, connection)
    to_write = [(str(tid),) for tid in pd.unique(pd.concat([nodes_df['tax_id'], names_df['tax_id']]))]
    insert_query = """
    INSERT INTO nodes (id) VALUES (?);
    """
    cursor.executemany(insert_query, to_write)
    connection.commit()


In [10]:
# make names table
with sqlite3.connect(db_path, autocommit=False) as connection:
    cursor = connection.cursor()
    create_table('names', names_schema, cursor, connection)
    to_write = [
        (str(uuid4()), str(tid), str(name)) for tid, name in 
        names_df[['tax_id', 'name']].to_records(index=False).tolist()
    ]
    insert_query = """
    INSERT INTO names (id, tax_id, name) VALUES (?, ?, ?)
    """
    cursor.executemany(insert_query, to_write)
    connection.commit()

In [12]:
# for unique names
unique_names = names_df.groupby('tax_id').agg({
    'name': lambda group: sorted(group, key=len)[0]
}).reset_index(drop=False)
with sqlite3.connect(db_path, autocommit=False) as connection:
    cursor = connection.cursor()
    create_table('names_unique', names_unique_schema, cursor, connection)
    to_write = [
        (str(uuid4()), str(tid), str(name)) for tid, name in 
        unique_names[['tax_id', 'name']].to_records(index=False).tolist()
    ]
    insert_query = """
    INSERT INTO names_unique (id, tax_id, name) VALUES (?, ?, ?)
    """
    cursor.executemany(insert_query, to_write)
    connection.commit()

In [41]:
# for parents table
col_names = ['tax_id', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus']
sql_col_names = ['id'] + ['t_' + x if x != 'tax_id' else x for x in col_names]
parents_df = pd.read_csv('../tax_parents.csv')[col_names]

with sqlite3.connect(db_path, autocommit=False) as connection:
    cursor = connection.cursor()
    create_table('parents', parents_schema, cursor, connection)
    to_write = [
        (str(uuid4()),) + tuple(e if not pd.isna(e) else None for e in record)
        for record in parents_df.to_records(index=False).tolist()
    ]
    insert_query = f"""
    INSERT INTO parents ({", ".join(sql_col_names)}) VALUES ({", ".join(["?" for _ in sql_col_names])})
    """
    cursor.executemany(insert_query, to_write)
    connection.commit()

In [14]:
# check
def query_df(query):
    with sqlite3.connect(db_path, autocommit=False) as connection:
        cursor = connection.cursor()
        cursor.execute(query)
        return cursor.fetchall()

In [43]:
query_df("""
SELECT * FROM names_unique WHERE tax_id = 5810
""")

[('da9df762-8cb4-4a24-bf99-17c03205e222', 5810, 'Toxoplasma')]